In [1]:
import psycopg2
import random
import requests
from numpy.random import choice

In [2]:
sectors = ['Private', 'Hotel', 'Gastronomy', 'Public']

In [3]:
conn = psycopg2.connect(dbname='admt_g02', user='admt_g02', password='xi6aeSoo', host='alcor.inf.unibz.it')
cur = conn.cursor()

In [4]:
def save_visitor(visitor, location_id, language, telephone):
    name = visitor['name']
    surname = visitor['surname']
    gender = visitor['gender']
    
    email = f'{name.lower()}.{surname.lower()}@email.com' 
    customer_number = f'\'{"".join([str(random.randint(0,9)) for _ in range(10)])}\'' if random.random() < 0.3 else 'NULL'
    
    sector = choice(sectors, p=[0.4, 0.3, 0.2, 0.1])
    
    #print(name, surname, gender, email, telephone, language, customer_number, sector, location_id)
    
    cur.execute(f'Insert into warehouse.visitor (visitor_name, visitor_telephone, visitor_email, \
    visitor_gender, visitor_language, location_id, visitor_sector, customer_number) values (\'{name} {surname}\', \'{telephone}\', \'{email}\', \
    \'{gender}\', \'{language}\', {location_id}, \'{sector}\', {customer_number})')

In [5]:
cur.execute('Delete from warehouse.visitor')

cur.execute('Select distinct district FROM warehouse.location where province = \'Bozen\'')
districts = cur.fetchall()

for district in districts:
    cur.execute('Select location_id FROM warehouse.location where district = \'%s\'' % district[0])
    cities = cur.fetchall()

    num_visitors_de = random.randint(20,30) if district[0] == 'Bozen' else random.randint(10,20)
    #print(district[0], num_visitors)
    
    res = requests.get(f'http://uinames.com/api/?region=austria&amount={num_visitors_de}')
    visitors_s_d = res.json()

    for visitor in visitors_s_d:
        location_id = cities[random.randrange(len(cities))][0]
        telephone = f'3{"".join([str(random.randint(0,9)) for _ in range(2)])} {"".join([str(random.randint(0,9)) for _ in range(7)])}'
        save_visitor(visitor, location_id, 'german', telephone)
        
    num_visitors_it = random.randint(20,30) if district[0] == 'Bozen' else random.randint(5,10)
    
    res = requests.get(f'http://uinames.com/api/?region=italy&amount={num_visitors_it}')
    visitors_s_i = res.json()

    for visitor in visitors_s_i:
        location_id = cities[random.randrange(len(cities))][0]
        telephone = f'3{"".join([str(random.randint(0,9)) for _ in range(2)])} {"".join([str(random.randint(0,9)) for _ in range(7)])}'
        save_visitor(visitor, location_id, 'italian', telephone)
        
    
    
    

In [6]:
cur.execute('Select location_id FROM warehouse.location where city is null and province = \'Trento\'')
provinces = cur.fetchall()

num_visitors_it = random.randint(20,30)

res = requests.get(f'http://uinames.com/api/?region=italy&amount={num_visitors_it}')
visitors_s_i = res.json()

for visitor in visitors_s_i:
    location_id = provinces[random.randrange(len(provinces))][0]
    telephone = f'3{"".join([str(random.randint(0,9)) for _ in range(2)])} {"".join([str(random.randint(0,9)) for _ in range(7)])}'
    save_visitor(visitor, location_id, 'italian', telephone)

In [7]:
cur.execute('Select location_id FROM warehouse.location where city is null and province is not null and province != \'Trento\'')
provinces = cur.fetchall()

num_visitors_it = random.randint(20,30)

res = requests.get(f'http://uinames.com/api/?region=italy&amount={num_visitors_it}')
visitors_s_i = res.json()

for visitor in visitors_s_i:
    location_id = provinces[random.randrange(len(provinces))][0]
    telephone = f'3{"".join([str(random.randint(0,9)) for _ in range(2)])} {"".join([str(random.randint(0,9)) for _ in range(7)])}'
    save_visitor(visitor, location_id, 'italian', telephone)

In [8]:
conn.commit()
cur.close()
conn.close()